In [11]:
pip install numpy matplotlib opencv-python tensorflow lxml

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     --------------------------------- ---- 41.0/46.2 kB 991.0 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 991.0 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 991.0 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/46.2 kB 991.0 kB/s eta 0:00:01
     --------------------------------------- 46.2/46.2 kB 65.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/377.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/377.0 MB 4.1 MB/s eta 0:01:32
   ---------------------------------------- 0.3/377.0 MB 4.0 MB/s eta 0:01:36
   ---------------------------------------- 0.3/377.0 MB 4.0 MB/s eta 0:01:36
   ---------------------------------------- 0.3/377.0 MB 4.0 MB/s eta 0:01:36
   ---------------------------------------- 0.3/377.0 MB 4.0 MB/s eta 0:01:36
   ---------------------------------------- 0.3/377.0 MB 4.0 MB/s e

In [ ]:
pip install sagemaker

: 

In [1]:
import numpy as np 
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from lxml import etree
import os
#
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#
print(cv2.__version__)
print(np.__version__)
print(plt.matplotlib.__version__)
print(tf.__version__)

4.9.0
1.26.4
3.8.3
2.16.1


In [2]:

paths= [os.path.join('assets','images', f) for f in sorted(os.listdir(r'.\assets\images'))]
classes = ['With Helmet', 'Without Helmet']
print(paths)

['assets\\images\\BikesHelmets0.png', 'assets\\images\\BikesHelmets1.png', 'assets\\images\\BikesHelmets10.png', 'assets\\images\\BikesHelmets100.png', 'assets\\images\\BikesHelmets101.png', 'assets\\images\\BikesHelmets102.png', 'assets\\images\\BikesHelmets103.png', 'assets\\images\\BikesHelmets104.png', 'assets\\images\\BikesHelmets105.png', 'assets\\images\\BikesHelmets106.png', 'assets\\images\\BikesHelmets107.png', 'assets\\images\\BikesHelmets108.png', 'assets\\images\\BikesHelmets109.png', 'assets\\images\\BikesHelmets11.png', 'assets\\images\\BikesHelmets110.png', 'assets\\images\\BikesHelmets111.png', 'assets\\images\\BikesHelmets112.png', 'assets\\images\\BikesHelmets113.png', 'assets\\images\\BikesHelmets114.png', 'assets\\images\\BikesHelmets115.png', 'assets\\images\\BikesHelmets116.png', 'assets\\images\\BikesHelmets117.png', 'assets\\images\\BikesHelmets118.png', 'assets\\images\\BikesHelmets119.png', 'assets\\images\\BikesHelmets12.png', 'assets\\images\\BikesHelmets12

In [3]:
images= []
labels= []
count=0
for image in paths:
    
    img_process= cv2.imread(image)
    img_process= np.array(img_process)
    img_process=  cv2.cvtColor(img_process, cv2.COLOR_BGR2GRAY)
    img_process= cv2.resize(img_process, (224, 224))
    (h, w) = img_process.shape[:2]
    images.append(img_process)
    
    count += 1
image_list= np.array(images)




In [3]:
def parse_annotation(annotation_file):
    tree = etree.parse(annotation_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text), int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text), int(bbox.find('ymax').text)]
        objects.append(obj_struct)
    return objects


def load_data(annotations_dir, images_dir):
    X, y, image_names = [], [], []
    for annotation_file in os.listdir(annotations_dir):
        if annotation_file.endswith('.xml'):
            annotation_path = os.path.join(annotations_dir, annotation_file)
            objects = parse_annotation(annotation_path)
            image_name = annotation_file.replace('.xml', '.png')  # Nome da imagem
            image_path = os.path.join(images_dir, image_name)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            X.append(preprocess_input(image))
            labels = [0] * len(classes)
            for obj in objects:
                label_idx = classes.index(obj['name'])
                labels[label_idx] = 1
            y.append(labels)
            image_names.append(image_name)  # Armazena o nome da imagem
    return np.array(X), np.array(y), image_names
images_dir=    r'assets\images'
annotations_dir=r'assets\annotations'
# Load and preprocess data
X_train, y_train, image_names_train = load_data(annotations_dir, images_dir)


In [8]:

# Define Faster R-CNN model
def create_faster_rcnn():
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(len(classes), activation='sigmoid')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    return model

# Compile model
model = create_faster_rcnn()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 139s 5s/step - accuracy: 0.6926 - loss: 0.6899 - val_accuracy: 0.7451 - val_loss: 0.4918
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8009 - loss: 0.4288 - val_accuracy: 0.7582 - val_loss: 0.4820
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8435 - loss: 0.3386 - val_accuracy: 0.7647 - val_loss: 0.4776
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8719 - loss: 0.2980 - val_accuracy: 0.7255 - val_loss: 0.5007
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8975 - loss: 0.2336 - val_accuracy: 0.8301 - val_loss: 0.4199
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.8933 - loss: 0.2178 - val_accuracy: 0.8105 - val_loss: 0.4778
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.9253 - loss: 0.1697 - val_accuracy: 0.8235 - val_loss: 0.4536
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.9285 - loss: 0.1561 - val_accuracy: 0.8105 - val_loss

In [12]:
model.save(r"assets\helmet_detection.h5")


In [17]:
load_model= tf.keras.models.load_model(r"assets\my_model.keras")
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

model.predict(preprocess_image(r'C:\Users\Erika Magalhães\Desktop\teste1.png'))  

c:\Users\Erika Magalhães\Desktop\helmet\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 218 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


array([[0.09351115, 0.947696  ]], dtype=float32)

In [4]:
import boto3
import sagemaker
cm_boto= boto3.client('sagemaker')
sess= sagemaker.Session()
region= sess.boto_session.region_name
bucket= 'mlbucketector'



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Erika Magalhães\AppData\Local\sagemaker\sagemaker\config.yaml
